<a href="https://colab.research.google.com/github/Veikko-Saikkonen/aalto-introduction-to-ml-for-materials-science/blob/main/Assignment_4_Tree_Methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 4: Tree methods

## Introduction

The purpose of this exercise is to test tree methods for regression and classification tasks.

The assignment consists of four exercises:

1. Building and visualizing a decision tree for regression of friction coefficients of different materials. (Essential content)
2. Building a random forest for regression of friction coefficients of different materials. (Essential Content)
3. Building a random forest for classification of the dimensionality of samples based on their X-ray diffraction (XRD) spectra. (Partly essential, partly optional content)
4. Building a gradient boosted forest for classification of the dimensionality of samples based on their XRD spectra. (Optional content)

The first exercise aims for introducing how tree-based decision making works and the second for introducing how the stability of the model is improved by building a forest of trees. In the last two exercises, two tree-based methods are compared in a cross-validation workflow that is close to real-life modelling workflow.

--

The friction coefficient data utilized in this exercise is originally published in (the article can be downloaded with Aalto VPN or in Aalto network):

Bucholz, E.W., Kong, C.S., Marchman, K.R. et al. Data-Driven Model for Estimation of Friction Coefficient Via Informatics Methods. Tribol Lett 47, 211–221 (2012). https://doi.org/10.1007/s11249-012-9975-y

The XRD data utilized in this exercise is originally published in:

Oviedo, F., Ren, Z., Sun, S. et al. Fast and interpretable classification of small X-ray diffraction datasets using data augmentation and deep neural networks. npj Comput Mater 5, 60 (2019). https://doi.org/10.1038/s41524-019-0196-x

The exercise structure is inspired by MIT PVLab's (Tonio Buonassisi's group) course exercises: https://github.com/PV-Lab/2s986_class (accessed 9/24/2022)


## Initial operations

Let's start by loading basic Python packages, installing and loading the X dataset, defining a data loader for XRD, and defining convenient plotting functions.

Run the cell below.

In [ ]:
import time  
from sklearn.metrics import accuracy_score
import numpy as np  
import matplotlib.pyplot as plt
import os

import pandas as pd
from scipy.signal import savgol_filter
from scipy.signal import find_peaks_cwt
from sklearn.model_selection import train_test_split

import random

## 1. Decision tree for regression (essential content)

### 1a. Import data

Let's start by initializing the data. Run the hidden cell below with a data loader defined.

In [ ]:
#@title Define a data loader.
def fric_data_loader():
    fric_data = pd.DataFrame([['MgO','Periclase',0.425,5.5,2,0.72,2.778,67.833,-1.747,-23.904,2.106,2.106,3098,1.31,3.44,2.13,3.6,40.304],
    ['SiO2','Quartz',0.449,7,4,0.4,10,44.728,-1.474,-52.773,1.5,1.61,1995,1.9,3.44,1.54,2.648,60.084],
    ['Al2O3','Corundum',0.4,9,3,0.54,5.556,56.709,-1.216,-28.334,1.327,1.855,2327,1.61,3.44,1.83,3.99,101.961],
    ['ZnO','Zincite',0.7,4,2,0.74,2.703,55.113,-1.642,-23.885,1.796,1.981,2247,1.65,3.44,1.79,5.6,81.38],
    ['CuO','Tenorite',0.4,3.5,2,0.77,2.597,44.728,-1.365,-20.199,1.277,1.948,1500,1.9,3.44,1.54,6.31,79.545],
    ['FeO','Wustite',0.6,5,2,0.55,3.636,47.692,-1.747,-23.350,2.155,2.155,1650,1.83,3.44,1.61,6,71.844],
    ['MoO3','Molybdite',0.235,3.5,6,0.69,8.696,33.608,-1.392,-61.521,2.102,1.956,1075,2.16,3.44,1.28,4.7,143.96],
    ['NiO','Bunsenite',0.5,5.5,2,0.69,2.899,44.302,-1.747,-24.150,2.084,2.084,2230,1.91,3.44,1.53,6.72,74.693],
    ['V2O5','Shcherbinaite',0.31,3.25,5,0.79,6.329,55.914,-1.486,-58.475,2.303,1.831,954,1.63,3.44,1.81,3.35,181.88],
    ['TiO2','Rutile',0.45,6.2,4,0.86,4.651,59.445,-1.600,-47.076,1.983,1.958,2116,1.54,3.44,1.9,4.17,79.866],
    ['SnO2','Cassiterite',0.5,6.5,4,0.69,5.797,42.166,-1.600,-44.890,2.057,2.054,1903,1.96,3.44,1.48,6.85,150.709],
    ['ZrO2','Baddeleyite',0.5,6.5,4,0.72,5.556,67.144,-1.660,-43.753,1.29,2.187,2983,1.33,3.44,2.11,5.68,123.223],
    ['Ag2S','Acanthite',0.101,2.3,1,1.15,0.87,10.024,-1.576,-8.921,2.072,2.546,1098,1.93,2.58,0.65,7.23,247.801],
    ['WS2','Tungstenite',0.043,2.5,4,0.6,6.667,1.203,-1.283,-30.666,3.124,2.411,1523,2.36,2.58,0.22,7.6,247.97],
    ['PbS','Galena',0.202,2.5,2,1.19,1.681,1.55,-1.747,-16.957,2.968,2.968,1386,2.33,2.58,0.25,7.6,239.3],
    ['Cu2S','Chalcocite',0.315,2.8,1,0.77,1.299,10.917,-1.567,-9.791,1.427,2.306,1402,1.9,2.58,0.68,5.6,159.157],
    ['MoS2','Molybdenite',0.22,1.3,4,0.69,5.797,4.314,-1.283,-30.486,2.98,2.425,1458,2.16,2.58,0.42,5.06,160.09],
    ['FeS2','Pyrite',0.2,6.3,2,0.55,3.636,13.118,-0.791,-10.070,1.464,2.264,1444,1.83,2.58,0.75,5.02,119.975],
    ['ZnS','Sphalerite',0.527,3.8,2,0.74,2.703,19.445,-1.637,-20.141,1.913,2.342,1973,1.65,2.58,0.93,4.04,97.44],
    ['Sb2S3','Stibnite',0.3,2,3,0.76,3.947,6.782,-1.551,-25.842,1.915,2.594,823,2.05,2.58,0.53,4.562,339.715],
    ['CdS','Greenockite',0.37,3.3,2,0.95,2.105,17.965,-1.642,-18.681,2.599,2.532,1753,1.69,2.58,0.89,4.826,144.476],
    ['NiS','Millerite',0.24,3.3,2,0.69,2.899,10.616,-1.626,-20.321,1.635,2.306,1249,1.91,2.58,0.67,5.5,90.758],
    ['MoSe2','Drysdallite',0.06,2,4,0.69,5.797,3.731,-1.283,-29.257,3.118,2.527,1473,2.16,2.55,0.39,6.9,253.88],
    ['ZnSe','Stilleite',0.49,5,2,0.74,2.703,18.331,-1.637,-19.222,2.004,2.454,1790,1.65,2.55,0.9,5.65,144.34],
    ['GaSe','P63/mmc',0.23,2,2,0.62,3.226,12.794,-1.039,-12.054,3.184,2.484,1233,1.81,2.55,0.74,5.03,148.68],
    ['CoSe','Freboldite',0.28,2.75,2,0.65,3.077,10.616,-1.706,-19.832,1.325,2.479,1328,1.88,2.55,0.67,7.65,137.89],
    ['Cu2Se','Berzelianite',0.49,2.7,1,0.77,1.299,10.024,-1.554,-8.855,1.46,2.529,1386,1.9,2.55,0.65,6.84,206.05],
    ['PbSe','Clausthalite',0.19,2.75,2,1.19,1.681,1.203,-1.747,-16.375,3.074,3.074,1351,2.33,2.55,0.22,8.1,286.2],
    ['CdTe','Zinc blende',0.718,3,2,0.95,2.105,4.115,-1.637,-16.810,2.291,2.806,1365,1.69,2.1,0.41,6.2,240.01],
    ['NiTe','Imgreite',0.28,4,2,0.69,2.899,0.898,-1.706,-18.566,1.339,2.648,1133,1.91,2.1,0.19,8.38,186.293],
    ['GaAs','Zinc blende',0.405,4.5,3,0.62,4.839,3.365,-2.455,-43.357,1.999,2.448,1511,1.81,2.18,0.37,5.318,144.645],
    ['CaF2','Fluorite',0.372,4,2,1,2,89.14,-0.839,-10.224,1.366,2.366,1691,1,3.98,2.98,3.18,78.075],
    ['BaF2','Frankdicksonite',0.392,2.5,2,1.35,1.481,90.81,-0.839,-9.009,1.55,2.685,1641,0.89,3.98,3.09,4.893,175.324],
    ['MgF2','Sellaite',0.429,5,2,0.72,2.778,83.174,-0.801,-11.583,1.981,1.992,1536,1.31,3.98,2.67,3.148,62.302],
    ['NaCl','Halite',0.303,2,1,1.02,0.98,71.155,-0.873,-4.461,2.82,2.82,1073.7,0.93,3.16,2.23,2.17,58.443],
    ['KCl','Sylvite',0.319,2,1,1.38,0.725,74.561,-0.873,-3.999,3.146,3.146,1044,0.82,3.16,2.34,1.988,74.551],
    ['KBr','Rock salt',0.379,1.5,1,1.38,0.725,68.174,-0.873,-3.813,3.3,3.3,1007,0.82,2.96,2.14,2.74,119.002],
    ['YPO4','Xenotime',0.357,4.5,3,0.9,3.333,52.884,-2.804,-51.689,2.243,2.345,2268,1.71,3.44,1.74,4.8,183.877]],
    columns = ['Chemical formula','Structure/phase','Friction coefficient','Mohs hardness','Formal cation charge',
               'Cation radius (Å)','Ionic potential','Percent ionicity (%)','Madelung constant',
               'Electrostatic potential (eV/atom)','Interplanar spacing (Å)','R ij distance (Å)',
               'Melting temperature (K)','Electronegativity of cation','Electronegativity of anion','EN difference','Density (g/cc)',
               'Molar weight (g/mol)'])
    
    fric = fric_data.loc[:, 'Friction coefficient'].values
    
    features_to_pick = [12,13,16]
    x = fric_data.iloc[:,features_to_pick].values
    
    columns_to_pick = [0,1,2] + features_to_pick
    
    fric_data = fric_data.iloc[:,columns_to_pick]
    
    return fric_data, fric, x

Let's load the data and familiarize ourselves with it. Fill in the gaps below.

In [ ]:
# Load the friction coefficient data.
raw_fric_data, fric, x = fric_data_loader()

# Print the first 10 rows of the raw_fric_data dataframe to get a general look at the data.

# Print out the shape of the dataframe and numpy arrays fric and x:


There are 38 samples in the dataframe. In this exercise, we aim for precting friction coefficients ("y data") as a function of other target properties. You can also see that there are non-numeric variables in the dataframe. For simplicity, we use only numeric variables as the input data for the models ("x data").

For convenience, the friction coefficients have already been saved into numpy array "fric" and the other numeric variables into numpy array "x". Fill in the gaps below.

In [ ]:
# Print variables fric and x.

# Print the number of samples in the friction coefficient data.
n_samples = None
print(n_samples)

# Print the number of variables (features) in the x fata.
n_dimensions = None
print(n_dimensions)

# Let's save and print the variable (feature) names in the x data
# (these need to be exactly same than column names in the dataframe).
x_names = None
print(x_names)

In [ ]:
#@title Check previous section
assert x_names[0] == 'Melting temperature (K)', 'x_names element 0 is not correctly defined, please check.'
assert x_names[1] == 'Electronegativity of cation', 'x_names element 1 is not correctly defined, please check.'
assert x_names[2] == 'Density (g/cc)', 'x_names element 2 is not correctly defined, please check.'
print("Yay ! All tests successsfully passed.")

### 1b. Decision tree regression for friction coefficient data

Let's fit a decision tree regressor model to predict friction coefficient based on materials properties in array "x".

Start by dividing the data into train and test sets. Fill in the gaps below.

In [ ]:
# Choose the number of samples in the train data.
n_train = None

# Remember how the data was shuffled and then divided
# into train and test in the exercise session last week?
# Let's do the same here (no need to do changes below).

# The rest of the data goes to test set.
n_test = n_samples - n_train

# shuffle the data
c = list(zip(x, fric))
random.shuffle(c)
x, frict = zip(*c)
x = np.array(x)
fric = np.array(fric)

# split data in training and test
# take first n_train samples for training
x_train  = x[0:n_train] 
fric_train = fric[0:n_train]

# take the next n_test data for testing
x_test = x[n_train:n_train + n_test]
fric_test = fric[n_train:n_train + n_test]

Let's create and fit a tree model with train data, and then use it for predicting test data. Finally, plot the regression result.

In [ ]:
# Let's import DecisionTreeRegressors.
from sklearn.tree import DecisionTreeRegressor

# Let's create a tree object.
model = DecisionTreeRegressor()
# Let's it it with train data.
model.fit(x_train, fric_train)

# Let's predict test data with the model.
fric_pred = model.predict(x_test)

# Let's plot the prediction result.
plt.figure(1)
# Let's plot predictions as the function of real values:
plt.plot(fric_test, fric_pred, 'x', color = 'b', label = 'Test dataset')
plt.ylabel('Predicted friction coeffiecient')
plt.xlabel('True friction coeffiecient')
# A perfect fit would match 1-to-1 and thus follow this line:
plt.plot([0, 1], [0, 1], 'k--', label = 'Perfect fit')
plt.legend()
plt.show()

Not that impressive, right? How does the tree make it decisions? Let's visualize the tree.

In [ ]:
from sklearn.tree import plot_tree

fig = plt.figure()
# Play around with this setting if the tree layout is not nice for your computer screen.
fig.set_size_inches(25, 15)
plot_tree(model, filled = True, feature_names = x_names, fontsize = 8)
plt.show()

In the graph above, if the decision criterion in a certain node is fulfilled ("True") for a certain sample, the sample goes down the left-side path. If the criterion is not fulfilled ("False"), the sample goes down the right-side path.

The color of the boxes has a meaning, too: it represents the high (dark color) or low (pale color) friction coefficient values flowing through the node.

Check how many samples there are assigned to the final leafs (nodes with no children) of the tree. Then, calculate how many train data samples have been assigned to the final leafs in the tree in total.

Now, what happens if we check the predictions on the _train_ data from this tree? Fill in the gaps.

In [ ]:
# Predict train data with the model.
fric_pred_on_train = None

# Let's plot the prediction result.
plt.figure()
# Let's plot test set predictions as the fucntion of real values:

# Plot here the train set predictions with red color:


plt.ylabel('Predicted friction coeffiecient')
plt.xlabel('True friction coeffiecient')
# A perfect fit would match 1-to-1 and thus follow this line:
plt.plot([0, 1], [0, 1], 'k--', label = 'Perfect fit')
plt.legend()
plt.show()

Perfect predictions for the train data! Seems rather surprising, taking into account that the test set predictions are poor.

But if you think about the tree illustration before, it is not such a surprise. Each leaf node was assigned to exactly one train datapoint. Thus, the train datapoint predictions always lead to these nodes and result in perfect values.

In the above graph, compare test datapoints carefully with train datapoints. You can zoom in in the above graph with 'plt.ylim([low, high])' and 'plt.xlim([low, high])' if you like.

The predictions of each test datapoint are exactly equal with the friction coefficient value of a corresponding train set datapoint! This means that in a single tree, the test datapoints cannot get any other values than those that were found from the train dataset. You can confirm this effect also by training a tree with a small train dataset (e.g. n_train = 3) - the test set prediction indeed get assigned discrete values from the train dataset friction coefficients.

### 1c. Hyperparameters of a decision tree -based model

Decision tree -based models have a lot of hyperparameters. Here, we test one of them: maximum depth of the tree (_max_depth_ in scikit-learn).

Calculate from the tree visualization above how many layers there were in the tree. Then try a lower value: Create a tree object with the chosen maximum depth given as a parameter, fit it, predict results for the test and train dataset, plot them both, and plot the tree visualization again.


In [ ]:
# Let's create a tree object with max_depth of 3 .
model = None
# Fit it it with train data.

# Predict test data with the model.

# Predict train data with the model.

# Plot the prediction result of train and test datasets.

# Plot the tree visualization.


Now the train dataset predictions are not exact anymore. This is because there are multiple samples grouped into the leaf nodes, and they all get assigned the same value.

## 2. Random forest regression (essential content)

### 2a. Random forest regression for friction coefficients data

Decision trees are visually interpretable models, but not very stable. In random forest regression, a forest of trees is trained to do the prediction and the final prediction is averaged among each tree. Let's fit a random forest regression model to predict the friction coefficient based on the descriptors in array 'x'. Run the cell below. Fill in the gaps.

In [ ]:
# Let's import RandomForestRegressor from sklearn.ensemble.
from sklearn.ensemble import RandomForestRegressor

# Let's create RF classifier model:
model = RandomForestRegressor()
# Fit the model with train data:

# Predict test data with the model.
fric_pred = None

# Predict train data with the model.
fric_pred_on_train = None

# Let's plot the prediction result.
plt.figure(1)
# Plot test set predictions as the function of real values:

plt.ylabel('Predicted friction coeffiecient')
plt.xlabel('True friction coeffiecient')
# A perfect fit would match 1-to-1 and thus follow this line:
plt.plot([0, 1], [0, 1], 'k--', label = 'Perfect fit')
plt.legend()
plt.show()

Looks somewhat better already with the default settings! How many trees there are in the random forest regression model above? Hint: check the documentation ( https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html ).

Random forest regression methods are difficult to visualize with the tree illustration because there are multiple trees in the forest. A common method to analyze the decision making of the model is feature importance ranking.

In [ ]:
# Feature importance ranking of x features:
print(model.feature_importances_)
print(x_names)

The more important the feature is, the higher the feature importance value. Check from the documentation what is the range of feature importance values. How important is the feature with the highest feature importance ranking compared to the other features in the train data? Or are they all equally important based on this ranking?

Note that the feature importance values tell about how important the feature is on average - it does not tell if high or low values of that feature lead to high values of the output.

--

In this exercise we used only three features for the regression model (for easier visualization) but you may remember from the class that the source article listed many more features. They are all loaded into this notebook, so nothing prevents you from training a random forest regression model on all the features and comparing to the model in the source article. But for now it is time to move on to XRD data!

## 3. Random forest classification of XRD data (essential and optional content)

Crystalline thin film samples may adopt different dimensionalities depending on their composition and sometimes even within fixed composition depending on their preparation methods. The dimensionality of the sample can be determined from its XRD spectrum but the analysis is time-consuming, especially if one needs to analyze large datasets. The purpose of this exercise is to divide the data into train and test sets, train a random forest classifier to automatically classify the XRD spectra based on their dimensionality using the train dataset, and finally test it on test dataset. This model could then be used as a pre-screening step for analyzing new XRD data.

### 3a. Preprocessing of XRD data (Essential content)

Let's start by initializing the data. Run the hidden cell below.

In [ ]:
#@title Define a raw data loader for XRD data.
def load_raw_data(from_file = False):
    
    if from_file:
        rawdata = pd.read_csv('./data/exp_d.csv',header=None)
    else:
        url = 'https://raw.githubusercontent.com/PV-Lab/2s986_class/master/Week7/data/exp_d.csv'
        rawdata = pd.read_csv(url, header = None)
        
    return rawdata


Let's define a data loader and plotting function.

In [ ]:
def data_loader():
    '''
    Data loader specifically for this exercise.
    '''
    
    # Load raw data.
    rawdata = load_raw_data()
    # Format the raw data as a numpy array.
    rawdata = rawdata.values
    rawdata[0,:] = [int(x[0]) for x in list(rawdata[0,:])]
    rawdata[1::,:] = rawdata[1::,:].astype(float)
    rawdata = np.delete(rawdata, 1, axis=0)
    # Save back to a dataframe.
    rawdata = pd.DataFrame(data = rawdata)
    rawdata = rawdata.rename(index={0: 'Dimensionality'})
    
    rawdata = rawdata.transpose()
    
    # XRD theta angles from the XRD spectra
    # (every second column in the dataframe but dropping the first row with the dimensionality labels):
    xrd_angles = rawdata.iloc[::2,1::].values
    
    # XRD intensities (same idea than for picking theta angles, but starting from the second column):
    xrd_intensities = rawdata.iloc[1::2, 1::].values

    # Class labels i.e. dimensionalities:
    dimensionalities = np.ravel(rawdata.iloc[::2,0].values).astype('int')
    
    return rawdata, dimensionalities, xrd_angles, xrd_intensities

def plot_xrd(theta_angles, intensities, colors = None, legend = None):
    '''
    Convenience function for plotting XRD and setting the axis labels.
    '''
    
    plt.plot(theta_angles.T, intensities.T, color = colors)
    
    plt.xlabel(r'2$\theta$ ($^{\circ}$)')
    plt.ylabel('Intensity (a.u.)')
    
    if legend is not None:
        plt.legend(legend)
    
    plt.tight_layout()


Let's load the data and familiarize ourselves with it.

In [ ]:
# Load the XRD spectra
rawdata, dimensionalities, xrd_angles, xrd_intensities = data_loader()

# Print out the shapes of dimensionalities, XRD angles, and XRD intensities arrays:


This dataset involves 73 samples with spectra with 3248 theta angles. This time, there is a theta vector for each sample - this is because the XRD spectra have been collected qith differing resolutions and/or ranges of theta.

In this dataset, we do not know the compositions of the samples. This is because the dataset has anonymized by the authors of the article it was published in (this is sometimes done due to copyright issues).

How do the XRD data look?

In [ ]:
# How does XRD data look? Save the XRD intensities of samples 0 and 20 into this variable and print them.
example_intensities = None

# Then save and print the theta angles of the example samples.
example_angles = None

Note that there are NaN values at the end of the vectors. The arrays have been padded with NaNs due to different lengths of the spectra.

Let's plot the example XRD spectra.

In [ ]:
# Let's plot the XRD intensities of the example samples as a function of XRD angle theta.
# For convenience, we use the function "plot_xrd" that has been defined above.
plt.figure(1)
plot_xrd(example_angles, example_intensities, legend = ['Sample 0', 'Sample 20'])
plt.show()

# Save the dimensionalities of the example samples 0 and 20 into this variable and print them.
example_dimensionalities = None

Well, at least these two samples with differing dimensionalities look quite different. We notice also two things:
- The intensity scale of the XRD spectra varies. You may notice from Tutorial 2 that this is due to the sample properties but also due to sample orientation and measurement device. The data needs to be normalized.
- We also notice that the spectra contain noise and background signal (especially visible in sample 0) that makes it more difficult to classify the data. We should remove background noise by filtering.

Let's start by the filtering task. Let's define a function to remove background noise (initially, this function is from the source article - you don't need to understand the steps), then apply it to the data.

Run the cell below.

In [ ]:
def exp_data_processing(data,minn,maxn,window):
    data = data.T
    (len1,w1) = np.shape(data)
    nexp1 =np.zeros([maxn-minn,w1])
    for i in range(w1):
        #savgol_filter to smooth the data
         new1 = savgol_filter(data[minn:maxn,i], 31, 3)
         #peak finding
         zf= find_peaks_cwt(new1, np.arange(10,15), noise_perc=0.01)
         #background substraction
         for j in range(len(zf)-1):
             zf_start= np.maximum(0,zf[j+1]-window//2)
             zf_end = np.minimum(zf[j+1]+window//2,maxn)
             peak = new1[zf_start:zf_end]

             ##arbitrarily remove 1/4 data
             npeak = np.maximum(0,peak-max(np.partition(peak,window//5 )[0:window//5]))
             nexp1[zf_start:zf_end,i]= npeak
                
    nexp1 = nexp1.T
    
    return nexp1

#window size for experimental data
window =15
#theta range
exp_min = 0
exp_max = 1350

# Clean up the data by filtering out the background.
filtered_intensities = exp_data_processing(xrd_intensities,exp_min,exp_max,window)
# The function also cut all the spectra to the same length so let's update theta angles, too.
filtered_angles = xrd_angles[:, exp_min:exp_max]

# Update the example spectra.
example_intensities = filtered_intensities[[0,20], :]
example_angles = filtered_angles[[0,20], :]

# Plot example sample 0 before and after the clean-up.
plt.figure(3)
plot_xrd(np.stack((example_angles[0,:], example_angles[0,:]), axis=0),
         np.stack((xrd_intensities[0, exp_min:exp_max], filtered_intensities[0,:]), axis = 0),
         legend = ['Sample 0 before clean-up', 'Sample 0 after clean-up'])
plt.show()

Looks better! Let's continue by normalizing the data. Run the cell below.

In [ ]:
from sklearn.preprocessing import minmax_scale
norm_intensities = minmax_scale(filtered_intensities, axis = 1)

example_intensities = norm_intensities[[0,20],:]

plt.figure(2)
plot_xrd(example_angles, example_intensities, legend = ['Sample 0', 'Sample 20'])
plt.show()

The data looks good now. Let's save the results to the final variables. Use these variables for XRD data from now on.

Run the cell below.

In [ ]:
cleaned_intensities = norm_intensities
cleaned_angles = filtered_angles

### 3b. Fitting random forest classifier model (Essential content)

Normally, you would perform hyperparameter optimization at this stage. This would be done by dividing the data randomly into train and test sets, and performing hyperparameter optimization using only the train data. Here, this step is omitted for the sake of time and we use a list of pre-defined hyperparameters below for the random forest classifier models. You can read more about hyperparameters of random forest models for example from here: https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

Run the cell below.

In [ ]:
'''
This code could be used as a starting point for hyperparameter optimization in your own projects:

random_grid = {'n_estimators': array([10, 25, 50, 75, 100, 500, 1000]),
 'max_depth': [1, 5, 10, 20, 50, 75, 100],
 'min_samples_split': [1, 2, 5, 10],
 'min_samples_leaf': [1, 2, 3, 4],
 'bootstrap': [True, False],
 'criterion': ['gini', 'entropy']}

model = RandomForestClassifier()
rf_random_cv = RandomizedSearchCV(estimator = model,
                               param_distributions = random_grid,
                               n_iter = 30, cv = 5,
                               verbose=2,
                               random_state=0)
rf_random_cv.fit(x_train, y_train)
'''

# Number of trees in the forest.
n_estimators = 100

# The maximum number of features considered for splitting a node.
max_features = 'sqrt'

# Maximum depth of the tree.
max_depth = None

Now, we divide the XRD data randomly into 5 folds of equal size. One fold is set as the test set, the rest are set as the train set.

In the first exercise and in the tutorial last week, we shuffled the data by ourselves, chose the train and test set sizes, and divided the data into two vectors. This process can also be done automatically in scikit-learn for example with the steps shown below.

Run the cell below.

In [ ]:
from sklearn.model_selection import KFold

# Number of folds. Means that the whole dataset is divided into n_fold parts of equal sizes.
n_fold = 5

# Create the KFold object.
k_fold = KFold(n_splits=n_fold, shuffle=True,random_state=0)

# Train and test indices for each fold:
train_test_idx_in_folds = {k: (train, test) for k, (train, test) in enumerate(k_fold.split(dimensionalities))}

# Let's print train set indices for each fold:
for i in range(n_fold):
    
    print('Train set indices for fold ' + str(i) + ': ', train_test_idx_in_folds[i][0])
    
    print('Test set indices for fold ' + str(i) + ': ', train_test_idx_in_folds[i][1], '\n')

You can notice that each test set index appears in the test set only in one fold, in other words, these folds do not overlap. There are also multiple other ways to define folds but this is typically a good starting point.

Let's use fold 0 train and test set as our train and test set for now.

In [ ]:
# Choose indices of train and test sets from fold 0:
fold = 0
train_idx = None
test_idx = None

# Pick input data for train and test sets (i.e., cleaned XRD intensities):
train_x = None
test_x = None

# Pick output data for train and test sets (i.e., sample dimensionalities):
train_y = None
test_y = None

Next, train a random forest _classification_ model for predicting the dimensionality of the XRD data based on the (cleaned-up) XRD intensities.

In [ ]:
# Import RandomForestClassifier from sklearn.ensemble.
from sklearn.ensemble import RandomForestClassifier

# Create RF classifier model with the hyperparameter settings you defined a bit earlier:
model = None
# Fit the model:

# Predict test set dimensionalities and print them out. Compare to the true dimensionalities.
pred_y = None

print('Predictions: ', None, '\nTrue values: ', None)

Now the random forest classifier is working but we still need to perform cross-validation to find out how well it performs.

### 3c. Cross-validation for performance evaluation (Optional content)

Turn the random forest classifier above into a function for easier use during cross-validation. Fill in:

In [ ]:
def my_RF_classifier(train_x, train_y, test_x):
    
    # Create RF classifier model.
    model = None
    # Fit the model:
    
    # Predict test set dimensionalities:
    pred_y = None
    
    return pred_y, model

Now we are ready for the cross validation. Fill in the gaps below.

In [ ]:
# This variable will be filled in during the for loop with cross validation scores.
accuracy = np.empty((n_fold,1)) 
# Let's time the for loop below.
start_time = time.time()   

for i in range(n_fold):
    
    # Define train and test sets for the current fold:
    
    # Use my_RF_classifier for fitting a random forest classifier and predicting the test set value with it.
    pred_y, model = my_RF_classifier(train_x, train_y, test_x)
    
    # Accuracy core for predicting the sample dimensionalities.
    accuracy[i] = accuracy_score(test_y, pred_y)  
    print ('Accuracy in fold ' + str(i) + ': %.2f%%' % (100 * accuracy[i]))
        
print ('Cross validation took %fs!' % (time.time() - start_time) )
print('Cross-validation results:')
print('Folds: %i, mean accuracy: %.3f' % (len(accuracy), np.mean(np.abs(accuracy))))


The random forest classifier was able to predict the dimensionality of the sample correct based on its XRD spectrum on average [mean accuracy]% of the tries during cross validation. This is good enough score for the random forest classifier to be useful in the actual use for pre-screening XRD data (but as always, you should also confirm the model predictions also by other means, for example by looking the spectra by yourself).

Next, you could proceed with more detailed cross validation to understand cases when your model performs well and when it does not. There are multiple different metrics for evaluating the predictions, and it is useful to compare their results. More information about typical classification accuracy metrics is for example here: https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics

Finally, you could either take any of the fold models and deploy it (default option), or use the same settings to train another random forest classifier with the whole dataset to be deployed (not a default option but is sometimes used with small datasets).

Look at the cross validation scores in each fold above. Some of them may perform clearly worse or better than the others. The variability depends on the random choice of the test and train dataset, and would likely be reduced with a larger source dataset. This could be done by collecting more experimental data. Another option, chosen by the authors of the source data article (link at the beginning of the exercise), is to further augment data to improve model performance.

## 4: Boosted tree classifier for XRD data (Optional content)

In the previous exercise you built a random forest regressor to predict the dimensionality of XRD data but is that the best possible type of model for the task? Typically, multiple different machine learning models (also different types of models) should be compared when building a regression or classification model because some models may perform better for a certain task.

In this exercise, you will compare the performance of the random forest classifier to a gradient boosting classifier.

Normally, you would optimize the hyperparameters for the gradient boosting classifier. For the sake of time, this step is omitted. Use the following hyperparameters, instead. Run the cell below. 

In [ ]:
# Number of trees in the forest.
n_estimators = 100

# The maximum number of features considered for splitting a node.
max_features = 'sqrt'

# Maximum depth of the tree.
max_depth = None

Define a function that creates a gradient boosting classifier, fits it to the train data, and makes prediction on the test data. Fill in the gaps.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

def my_boosted_tree_classifier(train_x, train_y, test_x):
    
    # Create classifier model with the hyperparameters defined above:
    model = None
    # Fit the model:
    
    # Predict test set dimensionalities:
    pred_y = None
    
    return pred_y, model

Repeat the cross validation in a same way than with the random forest classifier above.

Which one of the two models performed better? Which one was faster?

In on the tests you did here, the gradient boosting classifier likely performed slightly better than the random forest classifier but took a bit longer to train, so the gradient boosting classifier should be chosen for further use (for some choices of the train and test sets the two models may also perform similarly). 

Do you want to see how other types of models performed against the two tree-based models tested in this exercise? Check the source article for more comparisons.